In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

def create_db():
    conn = sqlite3.connect('todo.db')
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS tasks (
        id INTEGER PRIMARY KEY,
        task TEXT,
        category TEXT,
        date_time TEXT,
        completed INTEGER DEFAULT 0
    )
    ''')
    conn.commit()
    conn.close()

def add_task():
    task = task_entry.get()
    category = category_var.get()
    date_time = datetime_var.get()

    if task and date_time: 
        conn = sqlite3.connect('todo.db')
        c = conn.cursor()
        c.execute('INSERT INTO tasks (task, category, date_time) VALUES (?, ?, ?)', 
                  (task, category, date_time))
        conn.commit()
        conn.close()
        task_entry.delete(0, tk.END)
        datetime_entry.delete(0, tk.END)
        refresh_tasks()
    else:
        messagebox.showwarning("Warning", "The task and date/time cannot be empty.")

def refresh_tasks():
    tasks_listbox.delete(0, tk.END)
    conn = sqlite3.connect('todo.db')
    c = conn.cursor()
    c.execute('SELECT id, task, category, date_time, completed FROM tasks ORDER BY date_time, category')
    for row in c.fetchall():
        task = f"{row[0]}: {row[1]} [{row[2]}] {row[3]} - {'Completed' if row[4] else 'Pending'}"
        tasks_listbox.insert(tk.END, task)
    conn.close()

def delete_task():
    try:
        task_index = tasks_listbox.curselection()[0]
        task_text = tasks_listbox.get(task_index)
       
        task_id = int(task_text.split(":")[0])
        conn = sqlite3.connect('todo.db')
        c = conn.cursor()
        c.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
        conn.commit()
        conn.close()
        refresh_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "Please select a task to delete.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

def mark_as_complete():
    try:
        task_index = tasks_listbox.curselection()[0]
        task_text = tasks_listbox.get(task_index)
        task_id = int(task_text.split(":")[0])
        conn = sqlite3.connect('todo.db')
        c = conn.cursor()
        c.execute('UPDATE tasks SET completed = 1 WHERE id = ?', (task_id,))
        conn.commit()
        conn.close()
        refresh_tasks()
    except IndexError:
        messagebox.showwarning("Warning", "Please select a task to mark as complete.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

app = tk.Tk()
app.title("To-Do List Manager")

create_db()

# Task input
tk.Label(app, text="Task:").pack()
task_entry = tk.Entry(app, width=50)
task_entry.pack()

# Category selection
tk.Label(app, text="Category:").pack()
category_var = tk.StringVar()
category_combobox = ttk.Combobox(app, textvariable=category_var, values=["Work", "Home", "Other"])
category_combobox.pack()

# Date/Time input
tk.Label(app, text="Date/Time (YYYY-MM-DD HH:MM):").pack()
datetime_var = tk.StringVar()
datetime_entry = tk.Entry(app, textvariable=datetime_var, width=20)
datetime_entry.pack()

# Buttons
add_button = tk.Button(app, text="Add Task", command=add_task)
add_button.pack()

delete_button = tk.Button(app, text="Delete Selected Task", command=delete_task)
delete_button.pack()

complete_button = tk.Button(app, text="Mark as Complete", command=mark_as_complete)
complete_button.pack()

# Listbox for tasks
tasks_listbox = tk.Listbox(app, width=50, height=10)
tasks_listbox.pack()

refresh_tasks()

app.mainloop()
